In [192]:
import numpy as np
import pandas as pd
import scipy
import json
from collections import Counter
import copy

In [185]:
%%time
# 입력데이터 로드
train = pd.read_json('train.json', typ = 'frame')
genre = pd.read_json('genre_gn_all.json', typ = 'series')
meta = pd.read_json('song_meta.json', typ = 'frame')

CPU times: total: 15 s
Wall time: 15.1 s


In [179]:
genre_df = genre.to_frame().reset_index()
genre_df = genre_df.rename(columns = {"index":"genre_code",0:"genre_name"})

genre_code_name_dict = dict()
for _, row in genre_df.iterrows():
    genre_code_name_dict[row["genre_code"]] = row["genre_name"]

#genre_code_name_dict

In [176]:
def mapping_function(data, col1, col2):
    # 플레이리스트 아이디(col1)와 수록곡(col2) 추출
    plylst_song_map = data[[col1, col2]]

    # unnest col2
    plylst_song_map_unnest = np.dstack(
        (
            np.repeat(plylst_song_map[col1].values, list(map(len, plylst_song_map[col2]))),
            np.concatenate(plylst_song_map[col2].values)
        )
    )

    # unnested 데이터프레임 생성 : plylst_song_map
    plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
    plylst_song_map[col1] = plylst_song_map[col1].astype(str)
    plylst_song_map[col2] = plylst_song_map[col2].astype(str)

    # unnest 객체 제거
    del plylst_song_map_unnest
    return plylst_song_map

In [178]:
playlst_song_map = mapping_function(train, 'id', 'songs')
agg = pd.DataFrame(playlst_song_map['songs'].value_counts()).reset_index()
agg.columns = ['곡', '플레이리스트내의 등장횟수']
meta['id'] = meta['id'].astype(str)
agg = agg.merge(meta[['id', 'song_name', 'song_gn_gnr_basket']], how='left', left_on='곡', right_on='id')
agg.drop(['곡','id'], axis=1, inplace=True)
agg = agg.rename(columns={'song_gn_gnr_basket':'genre_code'})
agg = agg[agg["genre_code"].apply(lambda x: len(x) > 0)]

In [181]:
# 총 장르 수 254, 대분류 장르 30, 소분류 장르 224

gnr_code_list = list(genre.index) # 총 장르 list

gnr_code_name = pd.DataFrame(genre, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
large_gnr_code_list = gnr_code_name[gnr_code_name['gnr_code'].str[-2:] == '00'] # 대분류 장르 list

c = Counter()
for gnr_code in meta["song_gn_gnr_basket"]:
    if len(gnr_code) == 0: continue
    for sub_item in gnr_code:
        if sub_item in gnr_code_list:
            c.update([sub_item])

gnrname_number = pd.DataFrame.from_dict(c, orient='index').reset_index().rename(columns = {0:'number','index':'genre_name'})
gnrcode_number = pd.DataFrame.from_dict(c, orient='index').reset_index().rename(columns = {0:'number','index':'genre_code'})
gnrname_number["genre_name"] = gnrname_number["genre_name"].apply(lambda code: genre[code])
gnrcode_number = gnrcode_number.sort_values(by=["number"], ascending=[False]).reset_index(drop=True)
gnrname_number = gnrname_number.sort_values(by=["number"], ascending=[False]).reset_index(drop=True)
gnrname_number.index += 1
#gnrname_number.head(10) # 장르별 곡 수 (장르 랭킹)
#gnrcode_number.head(10)
gnrname_10 = gnrname_number.head(10)
#gnrname_10


top10_genre_code = gnrcode_number["genre_code"][:10].tolist()

In [187]:
genre_song_dict = dict()
for genre_code in top10_genre_code:
    genre_song_dict.setdefault(genre_code, [])

total = 0
for i, row in agg.iterrows():
    genre_code_list = row["genre_code"]
    song_name = row["song_name"]
    for g in genre_code_list:
        if not g in top10_genre_code: continue
        if len(genre_song_dict[g]) == 10: continue
        genre_song_dict[g].append(song_name)
        total += 1
    if total == 100: break

#genre_song_dict

In [189]:
genre_top10_song = pd.DataFrame(genre_song_dict)
genre_top10_song.index = genre_top10_song.index+1
genre_top10_song = genre_top10_song.T.reset_index()
genre_top10_song = genre_top10_song.rename(columns = {'index':'genre_code'})
genre_top10_song["genre_code"] = genre_top10_song["genre_code"].apply(lambda code: genre_code_name_dict[code])
genre_top10_song.index = genre_top10_song.index+1
genre_top10_song = genre_top10_song.rename(columns = {'genre_code': 'genre_name'})
#genre_top10_song

In [191]:
merged_df = genre_top10_song.merge(gnrname_10, on='genre_name', how='left')

desired_columns = ['genre_name', 'number']+[i for i in range(1,11)]
merged_df = merged_df[desired_columns]
merged_df.index = merged_df.index+1
merged_df

,genre_name,number,1,2,3,4,5,6,7,8,9,10
1,POP,77101,Lost Stars,I`m Not The Only One,Shape of You,YOUTH,Sugar,Uptown Funk (Feat. Bruno Mars),Love Yourself,Happy (Gru`s Theme From Despicable Me 2),One Call Away,Sunday Morning
2,록/메탈,63926,Lost Stars,Thinking Out Loud,Sugar,Fix You,Counting Stars,The Scientist,Photograph,Lucky (Feat. Colbie Caillat),Lucky Strike,Everglow
3,일렉트로니카,60144,Closer (Feat. Halsey),Something Just Like This,The Ocean (Feat. Shy Martin) (Radio Edit.),This Is What You Came For (Feat. Rihanna),Faded,How To Love (Feat. Sofia Reyes),"Feels (Feat. Pharrell Williams, Katy Perry & B...",Get Lucky (Feat. Pharrell Williams),Bad (Feat. Vassy) (Radio Edit),Slide (Feat. Frank Ocean & Migos)
4,클래식,57066,Debussy : Reverie L.68 (드뷔시 : 꿈),Provance,Grasse,Mozart : Piano Sonata No.9 In D Major K.311 - ...,Kreisler - Tambourin Chinois Op.3 (크라이슬러 : 중국의...,Kreisler - Kreisler Rachmaninov Liebesfreud (크...,Liszt : Grandes Etudes De Paganini No.6 In A M...,Satie : Gymnopedies No.1 - Lent Et Douloureux ...,10월의 어느 멋진 날에,Debussy : Children`s Corner L.113 - II. Jimbo`...
5,발라드,56820,밤편지,어떤이별,야생화,비가 내렸어 (Vocal by 스티브언니),벙어리,쉬운사랑,고마운 사람 (Vocal by 이소진),우연히 봄,금요일에 만나요 (Feat. 장이정 Of HISTORY),선물
6,재즈,48199,Don`t Know Why,Breezy,Home,"Quando, Quando, Quando",Sleeping Beauty,Soul Nights,Chocolate Drive,I Wish You Love,L - O - V - E,"March, April, May"
7,OST,44272,눈의 꽃,우연히 봄,Lost Stars,너였다면,너의 모든 순간,첫눈처럼 너에게 가겠다,너무 보고싶어,"모든 날, 모든 순간 (Every day, Every Moment)",만약에,걱정말아요 그대
8,뉴에이지,42515,Blue Sea,Happy Day,따뜻한 그날,Last Carnival,Fresh Morning,River Flows In You,바람따라 가는 기차여행,This Morning,Daily Life,Suddenly
9,인디음악,42468,안아줘,가끔 미치도록 네가 안고 싶어질 때가 있어,"그대와 나, 설레임 (Feat. 소울맨)",널 생각해,선물,사랑을 놓치고,넌 이별 난 아직 (With 한소현 Of 3rd Coast),스토커,"수고했어, 오늘도",불안해
10,랩/힙합,35376,See You Again (폴 워커 추모 엔딩곡),"I`m The One (Feat. Justin Bieber, Quavo, Chanc...",Nothin` On You (Feat. Bruno Mars),Empire State Of Mind (Feat. Alicia Keys),GDFR (Feat. Sage The Gemini And Lookas),HUMBLE.,Congratulations (Feat. Quavo),Caroline,The Monster (Feat. Rihanna),Panda
